### Running ANN using GPU

In [36]:
import torch

In [37]:
torch.cuda.is_available() # cuda is available in GPU

True

In [38]:
torch.cuda.current_device()

0

In [39]:
torch.cuda.get_device_name(0)

'GeForce GTX 1650'

In [40]:
torch.cuda.memory_cached()

C:\Users\Lenovo\anaconda3\lib\site-packages\torch\cuda\memory.py:440: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


2097152

In [41]:
torch.cuda.memory_allocated()

37376

In [42]:
var1 = torch.FloatTensor([1.0,2.0,3.0]).cuda()
var1

tensor([1., 2., 3.], device='cuda:0')

In [43]:
var1.device

device(type='cuda', index=0)

In [44]:
import pandas as pd
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
X=df.drop('Outcome',axis=1).values### independent features
y=df['Outcome'].values###dependent features

In [46]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [47]:
y_train

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [49]:
##### Creating Tensors
X_train=torch.FloatTensor(X_train).cuda()
X_test=torch.FloatTensor(X_test).cuda()
y_train=torch.LongTensor(y_train).cuda()
y_test=torch.LongTensor(y_test).cuda()

In [50]:
X_train.device

device(type='cuda', index=0)

In [51]:
#### Creating Modelwith Pytorch

class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [52]:
####instantiate my ANN_model
torch.manual_seed(20)
model=ANN_Model().cuda()

In [53]:
model.parameters

<bound method Module.parameters of ANN_Model(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=2, bias=True)
)>

In [54]:
for i in model.parameters():
    print(i.is_cuda)

True
True
True
True
True
True


In [55]:
###Backward Propogation-- Define the loss_function,define the optimizer
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [56]:
import time
start_time=time.time()
epochs=10000
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(time.time()-start_time)

Epoch number: 1 and the loss : 3.4572105407714844
Epoch number: 11 and the loss : 0.8019211292266846
Epoch number: 21 and the loss : 0.6090320348739624
Epoch number: 31 and the loss : 0.5917771458625793
Epoch number: 41 and the loss : 0.5679707527160645
Epoch number: 51 and the loss : 0.5529042482376099
Epoch number: 61 and the loss : 0.5410095453262329
Epoch number: 71 and the loss : 0.5310389995574951
Epoch number: 81 and the loss : 0.5220361948013306
Epoch number: 91 and the loss : 0.5135972499847412
Epoch number: 101 and the loss : 0.5061255097389221
Epoch number: 111 and the loss : 0.49834102392196655
Epoch number: 121 and the loss : 0.4960552155971527
Epoch number: 131 and the loss : 0.48286372423171997
Epoch number: 141 and the loss : 0.47560372948646545
Epoch number: 151 and the loss : 0.4799227714538574
Epoch number: 161 and the loss : 0.4793091416358948
Epoch number: 171 and the loss : 0.46500325202941895
Epoch number: 181 and the loss : 0.4582149386405945
Epoch number: 191 a

Epoch number: 1561 and the loss : 0.2868587076663971
Epoch number: 1571 and the loss : 0.269805371761322
Epoch number: 1581 and the loss : 0.2923907935619354
Epoch number: 1591 and the loss : 0.2629914879798889
Epoch number: 1601 and the loss : 0.2832219898700714
Epoch number: 1611 and the loss : 0.29477787017822266
Epoch number: 1621 and the loss : 0.2591519057750702
Epoch number: 1631 and the loss : 0.2605319321155548
Epoch number: 1641 and the loss : 0.2743358314037323
Epoch number: 1651 and the loss : 0.25780603289604187
Epoch number: 1661 and the loss : 0.26152297854423523
Epoch number: 1671 and the loss : 0.2562791705131531
Epoch number: 1681 and the loss : 0.277321457862854
Epoch number: 1691 and the loss : 0.27466708421707153
Epoch number: 1701 and the loss : 0.2728791832923889
Epoch number: 1711 and the loss : 0.25448670983314514
Epoch number: 1721 and the loss : 0.2557942271232605
Epoch number: 1731 and the loss : 0.27429986000061035
Epoch number: 1741 and the loss : 0.273846

Epoch number: 3211 and the loss : 0.17704612016677856
Epoch number: 3221 and the loss : 0.18323658406734467
Epoch number: 3231 and the loss : 0.18237732350826263
Epoch number: 3241 and the loss : 0.18147999048233032
Epoch number: 3251 and the loss : 0.19616127014160156
Epoch number: 3261 and the loss : 0.18194754421710968
Epoch number: 3271 and the loss : 0.19623246788978577
Epoch number: 3281 and the loss : 0.175063818693161
Epoch number: 3291 and the loss : 0.17320644855499268
Epoch number: 3301 and the loss : 0.17661446332931519
Epoch number: 3311 and the loss : 0.18374516069889069
Epoch number: 3321 and the loss : 0.18514373898506165
Epoch number: 3331 and the loss : 0.20281866192817688
Epoch number: 3341 and the loss : 0.37165704369544983
Epoch number: 3351 and the loss : 0.2781924307346344
Epoch number: 3361 and the loss : 0.20002540946006775
Epoch number: 3371 and the loss : 0.18469828367233276
Epoch number: 3381 and the loss : 0.18703284859657288
Epoch number: 3391 and the loss

Epoch number: 4741 and the loss : 0.1272967904806137
Epoch number: 4751 and the loss : 0.12624354660511017
Epoch number: 4761 and the loss : 0.12517796456813812
Epoch number: 4771 and the loss : 0.1241564080119133
Epoch number: 4781 and the loss : 0.12335806339979172
Epoch number: 4791 and the loss : 0.12264424562454224
Epoch number: 4801 and the loss : 0.1218237578868866
Epoch number: 4811 and the loss : 0.12109337747097015
Epoch number: 4821 and the loss : 0.12065406888723373
Epoch number: 4831 and the loss : 0.12018687278032303
Epoch number: 4841 and the loss : 0.1190517470240593
Epoch number: 4851 and the loss : 0.1184849962592125
Epoch number: 4861 and the loss : 0.1177048459649086
Epoch number: 4871 and the loss : 0.11710882186889648
Epoch number: 4881 and the loss : 0.11653434485197067
Epoch number: 4891 and the loss : 0.11598210781812668
Epoch number: 4901 and the loss : 0.11607234179973602
Epoch number: 4911 and the loss : 0.1151605024933815
Epoch number: 4921 and the loss : 0

Epoch number: 6331 and the loss : 0.13884718716144562
Epoch number: 6341 and the loss : 0.12688767910003662
Epoch number: 6351 and the loss : 0.12811695039272308
Epoch number: 6361 and the loss : 0.12362676113843918
Epoch number: 6371 and the loss : 0.12612324953079224
Epoch number: 6381 and the loss : 0.1309366524219513
Epoch number: 6391 and the loss : 0.25646984577178955
Epoch number: 6401 and the loss : 0.15610015392303467
Epoch number: 6411 and the loss : 0.13152875006198883
Epoch number: 6421 and the loss : 0.14222452044487
Epoch number: 6431 and the loss : 0.12989874184131622
Epoch number: 6441 and the loss : 0.19766028225421906
Epoch number: 6451 and the loss : 0.14971841871738434
Epoch number: 6461 and the loss : 0.13179410994052887
Epoch number: 6471 and the loss : 0.14688532054424286
Epoch number: 6481 and the loss : 0.14267705380916595
Epoch number: 6491 and the loss : 0.1446133404970169
Epoch number: 6501 and the loss : 0.17115478217601776
Epoch number: 6511 and the loss :

Epoch number: 7861 and the loss : 0.10223709046840668
Epoch number: 7871 and the loss : 0.1016126275062561
Epoch number: 7881 and the loss : 0.10161711275577545
Epoch number: 7891 and the loss : 0.10359212011098862
Epoch number: 7901 and the loss : 0.10103436559438705
Epoch number: 7911 and the loss : 0.1016467958688736
Epoch number: 7921 and the loss : 0.09947283565998077
Epoch number: 7931 and the loss : 0.12008868157863617
Epoch number: 7941 and the loss : 0.5895192623138428
Epoch number: 7951 and the loss : 0.38226380944252014
Epoch number: 7961 and the loss : 0.30368587374687195
Epoch number: 7971 and the loss : 0.2086855173110962
Epoch number: 7981 and the loss : 0.15452295541763306
Epoch number: 7991 and the loss : 0.14172835648059845
Epoch number: 8001 and the loss : 0.12766635417938232
Epoch number: 8011 and the loss : 0.12096626311540604
Epoch number: 8021 and the loss : 0.11809934675693512
Epoch number: 8031 and the loss : 0.11586899310350418
Epoch number: 8041 and the loss 

Epoch number: 9441 and the loss : 0.086729496717453
Epoch number: 9451 and the loss : 0.0863412395119667
Epoch number: 9461 and the loss : 0.08647049218416214
Epoch number: 9471 and the loss : 0.08590737730264664
Epoch number: 9481 and the loss : 0.08586413413286209
Epoch number: 9491 and the loss : 0.08570186793804169
Epoch number: 9501 and the loss : 0.08518961071968079
Epoch number: 9511 and the loss : 0.08505494892597198
Epoch number: 9521 and the loss : 0.08495967835187912
Epoch number: 9531 and the loss : 0.08477713912725449
Epoch number: 9541 and the loss : 0.08433572947978973
Epoch number: 9551 and the loss : 0.0841713473200798
Epoch number: 9561 and the loss : 0.08409325778484344
Epoch number: 9571 and the loss : 0.08372491598129272
Epoch number: 9581 and the loss : 0.08480313420295715
Epoch number: 9591 and the loss : 0.08338871598243713
Epoch number: 9601 and the loss : 0.08329702913761139
Epoch number: 9611 and the loss : 0.13353702425956726
Epoch number: 9621 and the loss 